In [1]:
# Instala o banco de dados vetorial ChromaDB
!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/24/2a/5cad8436a39c45f9fb9a6a0662d6254039f90a5710cf82410b685bbb7c71/chromadb-0.4.17-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic>=1.9 from https://files.pythonhosted.org/packages/e2/2c/9906b7abc337b0250a5634de5396e2f3cb1b837af0616424c2225a65aa80/pydantic-2.5.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 416.8 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/48/0e/068b658a547d6090b969014146321e28dae1411da54b76d081e51a2af22b/chroma_hnswlib-0.7.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/f3/4f/0ce34195b63240b6693086496c9bab4ef23999112184399a3e88854c7674/fastapi-0.104.1-py3-none-any.whl.metadata
  Obtaining dependency informat

In [2]:
# Instala a biblioteca que possibilita a manipulação de LLMs
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 685.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 108.8 kB/s eta 0:00:0000:0100:01
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/f2/b8/b1ec82fce93064a73ba67f2bb158ec9cac4a0e8f0b6942268ec963947329/regex-2023.10.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 355.5 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency informatio

In [3]:
import pandas as pd
import networkx as nx
import chromadb
from sentence_transformers import SentenceTransformer

1) Os dados disponibilizados representam notícias coletados pela agência. Inicialmente estes devem ser carregados e pré-processados, por exemplo, eliminado o conteúdo "\n" do corpo de texto de cada notícia. Ou outro pré-processamento que seja requerido. Os dados de todos os arquivos disponibilizados devem ser carregados;


In [40]:
#url do dataset completo: https://github.com/fergiemcdowall/reuters-21578-json/tree/master/data/full
# O campo body deve passar por um pré-processamento alterando \n por espaço

import os 
import json

'''
file_name = '/workspaces/DataScience_class/atividades/Avaliacao II/reuters-000.json'

import json

data = json.load(open(file_name, "r"))
df_list = list()
for item in data:
  df_list.append(item)

df = pd.DataFrame(df_list)
df
'''
def json_files(get_path = None):
    #Substitua path pela sua pasta onde estão os arquivos json
    path = get_path
    list_paths = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.json'):
                list_paths.append(os.path.join(root, file))

    # Criando um dataframe vazio
    df = pd.DataFrame()
    list_paths.sort()
    # Loop para carregar os arquivos json e concatenar no dataframe
    for path in list_paths:
        # Carregando o arquivo json
        data = json.load(open(path, "r"))

        # Criando um dataframe com o arquivo json
        df_list = list()
        for item in data:
            df_list.append(item)
        df_temp = pd.DataFrame(df_list)
        # Concatenando o dataframe criado com os dataframes anteriores
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

df = json_files("/workspaces/DataScience_class/atividades/Avaliacao II")
df.head()


,title,body,date,topics,places,id,organisations
0,BAHIA COCOA REVIEW,Showers continued throughout the week in\nthe ...,26-FEB-1987 15:01:01.79,[cocoa],"[el-salvador, usa, uruguay]",1,NaN
1,STANDARD OIL <SRD> TO FORM FINANCIAL UNIT,Standard Oil Co and BP North America\nInc said...,26-FEB-1987 15:02:20.00,NaN,[usa],2,NaN
2,TEXAS COMMERCE BANCSHARES <TCB> FILES PLAN,Texas Commerce Bancshares Inc's Texas\nCommerc...,26-FEB-1987 15:03:27.51,NaN,[usa],3,NaN
3,TALKING POINT/BANKAMERICA <BAC> EQUITY OFFER,BankAmerica Corp is not under\npressure to act...,26-FEB-1987 15:07:13.72,NaN,"[usa, brazil]",4,NaN
4,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The U.S. Agriculture Department\nreported the ...,26-FEB-1987 15:10:44.60,"[grain, wheat, corn, barley, oat, sorghum]",[usa],5,NaN


In [46]:
#!pip install rapids
#tratando os dados
df = df[['title','body']]
df = df.dropna()
df = df.reset_index(drop=True)
#retira os \n do body
df['body'] = df['body'].str.replace('\n',' ')
df.head()



The total number of data-points after removing the rows with missing values are: 4584


In [11]:
# Cria uma instância do SBERT considerando o LLM all-MiniLM-L6-v2
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

2) Cada notícia, após o pré-processamento, deve ser transformada em um embedding considerando a concatenação do título e corpo da notícia;

In [51]:
#doc = 'Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century.'
#2) Cada notícia, após o pré-processamento, deve ser transformada em um embedding considerando a concatenação do título e corpo da notícia.

# Cria uma função para gerar o embedding de uma notícia e o corpo da notícia
def embedding_news(news):
  # Concatena o título e o corpo da notícia
  text = news['title'] + ' ' + news['body']
  # Gera o embedding da notícia
  embedding = embedding_model.encode(text)
  # Retorna o embedding da notícia
  return embedding

# Cria uma função para gerar o embedding de todas as notícias
def embedding_all_news(df):
  # Cria uma lista vazia para armazenar os embeddings
  embeddings = []
  # Loop para gerar o embedding de cada notícia
  for index, row in df.iterrows():
    # Gera o embedding da notícia
    embedding = embedding_news(row)
    # Armazena o embedding na lista de embeddings
    embeddings.append(embedding)
  # Retorna a lista de embeddings
  return embeddings

# Gera o embedding de todas as notícias
embeddings = embedding_all_news(df)
embeddings[0]





array([ 6.32261578e-03, -3.46379131e-02, -2.23485613e-03,  7.46762082e-02,
        1.78895332e-02, -2.90669054e-02, -1.29468879e-02,  3.30298543e-02,
       -6.97685108e-02, -4.33890149e-03,  3.17571200e-02, -1.34216130e-01,
       -7.67679214e-02, -1.66306384e-02, -4.18836959e-02, -7.76778953e-03,
        3.57857682e-02, -1.41272638e-02, -2.48942655e-02,  4.97413194e-03,
        3.19821090e-02, -2.05820557e-02, -2.37906277e-02,  2.78013870e-02,
       -3.26795541e-02,  8.69515836e-02, -6.30176663e-02, -1.31031200e-01,
        1.37793575e-03, -4.20760782e-03, -6.55597448e-02,  5.43250702e-02,
        9.36558619e-02,  2.79298946e-02,  3.23991440e-02,  2.10880861e-03,
        5.33334315e-02, -1.68298855e-02,  3.50959189e-02, -9.26825218e-04,
        4.20695208e-02,  2.97183134e-02, -2.42490824e-02,  3.19630392e-02,
        2.48613320e-02, -6.36065612e-03,  6.55190423e-02,  6.57907277e-02,
        1.66355260e-02,  5.42933447e-03,  3.25612649e-02,  6.80907369e-02,
       -9.92129222e-02,  

3) Pensando em um ambiente real e, evitando a geração frequente de embeddings, estes deve ser armazenados em um banco de dados vetorial;

In [57]:
# Cria uma instância do ChromaDB
client = chromadb.PersistentClient(path="/workspaces/DataScience_class/database")
collection = client.create_collection("collection")




UniqueConstraintError: Collection collection already exists

In [58]:
# inserir o id, o embedding e o texto da notícia no ChromaDB
for index, row in df.iterrows():
  # Gera o embedding da notícia
  embedding = embeddings[index]
  # Insere o documento no ChromaDB
  collection.add({
      ids= [index],
      embeddings= embedding
  })



ValueError: Expected IDs to be a list, got [ 6.32261578e-03 -3.46379131e-02 -2.23485613e-03  7.46762082e-02
  1.78895332e-02 -2.90669054e-02 -1.29468879e-02  3.30298543e-02
 -6.97685108e-02 -4.33890149e-03  3.17571200e-02 -1.34216130e-01
 -7.67679214e-02 -1.66306384e-02 -4.18836959e-02 -7.76778953e-03
  3.57857682e-02 -1.41272638e-02 -2.48942655e-02  4.97413194e-03
  3.19821090e-02 -2.05820557e-02 -2.37906277e-02  2.78013870e-02
 -3.26795541e-02  8.69515836e-02 -6.30176663e-02 -1.31031200e-01
  1.37793575e-03 -4.20760782e-03 -6.55597448e-02  5.43250702e-02
  9.36558619e-02  2.79298946e-02  3.23991440e-02  2.10880861e-03
  5.33334315e-02 -1.68298855e-02  3.50959189e-02 -9.26825218e-04
  4.20695208e-02  2.97183134e-02 -2.42490824e-02  3.19630392e-02
  2.48613320e-02 -6.36065612e-03  6.55190423e-02  6.57907277e-02
  1.66355260e-02  5.42933447e-03  3.25612649e-02  6.80907369e-02
 -9.92129222e-02  1.69798099e-02 -2.95887375e-03 -4.23186161e-02
  1.93335116e-02 -8.05187225e-02  4.49305028e-0